# Importing libraries and creating global values

In [326]:
HighestBranchId = 0
Variables = {}

# Variables



In [327]:
class Token:
    def __init__(self, tokenType, value, position=[1, 1]):
        self.value = value
        self.tokenType = tokenType
        self.position = position # [line position, starting character position]


    def __str__(self):
        return f"{self.tokenType} {self.value} {self.position}"

    def __repr__(self):
        return f"{self.tokenType} {self.value} {self.position}"

In [328]:
class FluoryVariable:
    def __init__(self, id, expression, varType):
        self.expression = expression # Tokens
        self.id = id
        self.value = 0
        self.parentVariables = []
        self.childVariables = []
        self.varType = varType

    def __str__(self):
        return str(self.value)

    def __repr__(self):
        return str(self.value)

    def setSelf(self, tokens):
        self.expression = tokens

    def resolveSelf(self):
        global Variables
        self.value = self.resolve(self.expression)
        Variables[self.id] = self

    def resolve(self, tokens):
        raise Exception("Warning: This variable type has no self resolving feature. Value will stay 0, please contact your favorite programming language developper to resolv this issue.")
        self.value = None

    def add(self, firstVal, secondVal):
        return firstVal + secondVal

    def sub(self, firstVal, secondVal):
        return firstVal - secondVal

    def mul(self, firstVal, secondVal):
        return firstVal * secondVal

    def div(self, firstVal, secondVal):
        return firstVal / secondVal

    def mod(self, firstVal, secondVal):
        return firstVal % secondVal

    def pow(self, firstVal, secondVal):
        return firstVal ** secondVal

    def floorDiv(self, firstVal, secondVal):
        return firstVal // secondVal

    def orLogic(self, firstVal, secondVal):
        return firstVal | secondVal

    def andLogic(self, firstVal, secondVal):
        return firstVal & secondVal

    def xorLogic(self, firstVal, secondVal):
        return firstVal ^ secondVal

    def notLogic(self, val):
        return not val

    def toInt(self):
        return int(self.value)

    def toFloat(self):
        return float(self.value)

    def toString(self):
        return str(self.value)

    def toBool(self):
        return bool(self.value)

    def readNextToken(self, cursor, tokens):
        if cursor >= len(tokens):
            raise Exception("Error: Unexpected end of file")

        cursor += 1
        currentToken = tokens[cursor]

        return cursor, currentToken

    def peakNextToken(self, cursor, tokens):
        if cursor+1 >= len(tokens):
            raise Exception("Error: Unexpected end of file")

        return tokens[cursor+1]

    def separateTokensOn(self, key, cursor, tokens):
        if cursor >= len(tokens):
            raise Exception("Error: Unexpected end of file")

        separatedTokens = []

        while tokens[cursor].value != key:
            if cursor >= len(tokens):
                raise Exception("Error: Unexpected end of file")

            separatedTokens.append(tokens[cursor])
            cursor += 1

        return cursor+1, separatedTokens

    def updateParentVariables(self, newParentVariables):
        global Variables

        newParentVariables = set(newParentVariables)

        bondsToRemove = list(set(self.parentVariables) - newParentVariables)
        bondsToAdd = list(newParentVariables - set(self.parentVariables))

        for bond in bondsToRemove:
            Variables[bond].childVariables.remove(self.id)

        for bond in bondsToAdd:
            Variables[bond].childVariables.append(self.id)

        self.parentVariables = list(newParentVariables)

    def updateChildVariables(self):
        global Variables

        for child in self.childVariables:
            Variables[child].resolveSelf()


In [329]:
class FluoryInteger(FluoryVariable):
    def __init__(self, id, value):
        super().__init__(id, value, "int")

    def resolve(self, tokens):
        global Variables

        # We will need to keep track on what other variables we are reliant on
        newParentVariables = []

        currentValue = 0
        # First value must be a value
        if tokens[0].tokenType in ["int", "float", "str", "bool"]:
            currentValue = int(tokens[0].value)

        # Turns out it will be a value soon
        elif tokens[0].value == "(":
            cursor, separatedTokens = self.separateTokensOn(")", 1, tokens)
            currentValue = self.resolve(separatedTokens)

        # Maybe its a value, its just hiding
        elif tokens[0].tokenType == "variableCall":
            if tokens[0].value in Variables:
                otherVar = Variables[tokens[0].value]

                currentValue = Variables[tokens[0].value].toInt()
                newParentVariables.append(tokens[0].value)

            else:
                raise Exception(f"Error: Variable '{tokens[0].value}' is undeclared")

        # Ok, its not a value after all
        else:
            raise Exception(f"Error: Expected integer but got '{tokens[0].tokenType}' at line {tokens[0].position[0]}, character {tokens[0].position[1]}")

        cursor=0

        while cursor+1 < len(tokens):
            #This should be our next number
            cursor, currentToken = self.readNextToken(cursor, tokens)
            nextToken = self.peakNextToken(cursor, tokens)

            # Verifying that current value is indded an operation
            if currentToken.tokenType != "arithmeticOperation":
                raise Exception(f"Error: Expected arithmetic operation but got '{currentToken.tokenType}' at line {currentToken.position[0]}, character {currentToken.position[1]}")

            # Init number
            upcomingValue = 0

            # Is it a number?
            if nextToken.tokenType in ["int", "float", "str", "bool"]:
                upcomingValue = int(nextToken.value)
                cursor+=1

            # Its going to be a number!!
            elif nextToken.value == "(":
                cursor, separatedTokens = self.separateTokensOn(")", cursor+2, tokens)
                upcomingValue = self.resolve(separatedTokens)

            # Maybe its a number, its just hiding
            elif nextToken.tokenType == "variableCall":
                if nextToken.value in Variables:
                    otherVar = Variables[nextToken.value]

                    upcomingValue = otherVar.toInt()
                    newParentVariables.append(nextToken.value)
                    cursor+=1

                else:
                    raise Exception(f"Error: Variable '{nextToken.value}' is undeclared")

            # Turns out it was not a number after all..
            else:
                raise Exception(f"Error: Expected integer but got '{nextToken.tokenType}' at line {tokens[0].position[0]}, character {tokens[0].position[1]}")

            # Doing math!
            match currentToken.value:
                case "+":  currentValue = self.add(currentValue, upcomingValue)
                case "-":  currentValue = self.sub(currentValue, upcomingValue)
                case "*":  currentValue = self.mul(currentValue, upcomingValue)
                case "/":  currentValue = self.div(currentValue, upcomingValue)
                case "%":  currentValue = self.mod(currentValue, upcomingValue)
                case "**": currentValue = self.pow(currentValue, upcomingValue)
                case "//": currentValue = self.floorDiv(currentValue, upcomingValue)
                case _ :   raise Exception(f"Error: Unknown operation '{currentToken.value}' at line {currentToken.position[0]}, character {currentToken.position[1]}")


        self.updateParentVariables(newParentVariables)

        return int(currentValue)


In [330]:
class FluoryFloat(FluoryVariable):
    def __init__(self, id, value):
        super().__init__(id, value, "float")

    def resolve(self, tokens):
        global Variables

        # We will need to keep track on what other variables we are reliant on
        newParentVariables = []

        currentValue = 0
        # First value must be a value
        if tokens[0].tokenType in ["int", "float", "str", "bool"]:
            currentValue = float(tokens[0].value)

        # Turns out it will be a value soon
        elif tokens[0].value == "(":
            cursor, separatedTokens = self.separateTokensOn(")", 1, tokens)
            currentValue = self.resolve(separatedTokens)

        # Maybe its a value, its just hiding
        elif tokens[0].tokenType == "variableCall":
            if tokens[0].value in Variables:
                otherVar = Variables[tokens[0].value]

                currentValue = Variables[tokens[0].value].toFloat()
                newParentVariables.append(tokens[0].value)

            else:
                raise Exception(f"Error: Variable '{tokens[0].value}' is undeclared")

        # Ok, its not a value after all
        else:
            raise Exception(f"Error: Expected floating point but got '{tokens[0].tokenType}' at line {tokens[0].position[0]}, character {tokens[0].position[1]}")

        cursor=0

        while cursor+1 < len(tokens):
            #This should be our next number
            cursor, currentToken = self.readNextToken(cursor, tokens)
            nextToken = self.peakNextToken(cursor, tokens)

            # Verifying that current value is indded an operation
            if currentToken.tokenType != "arithmeticOperation":
                raise Exception(f"Error: Expected arithmetic operation but got '{currentToken.tokenType}' at line {currentToken.position[0]}, character {currentToken.position[1]}")

            # Init number
            upcomingValue = 0

            # Is it a number?
            if nextToken.tokenType in ["int", "float", "str", "bool"]:
                upcomingValue = float(nextToken.value)
                cursor+=1

            # Its going to be a number!!
            elif nextToken.value == "(":
                cursor, separatedTokens = self.separateTokensOn(")", cursor+2, tokens)
                upcomingValue = self.resolve(separatedTokens)

            # Maybe its a number, its just hiding
            elif nextToken.tokenType == "variableCall":
                if nextToken.value in Variables:
                    otherVar = Variables[nextToken.value]

                    upcomingValue = otherVar.toFloat()
                    newParentVariables.append(nextToken.value)
                    cursor+=1

                else:
                    raise Exception(f"Error: Variable '{nextToken.value}' is undeclared")

            # Turns out it was not a number after all..
            else:
                raise Exception(f"Error: Expected floating point but got '{nextToken.tokenType}' at line {tokens[0].position[0]}, character {tokens[0].position[1]}")

            # Doing math!
            match currentToken.value:
                case "+":  currentValue = self.add(currentValue, upcomingValue)
                case "-":  currentValue = self.sub(currentValue, upcomingValue)
                case "*":  currentValue = self.mul(currentValue, upcomingValue)
                case "/":  currentValue = self.div(currentValue, upcomingValue)
                case "%":  currentValue = self.mod(currentValue, upcomingValue)
                case "**": currentValue = self.pow(currentValue, upcomingValue)
                case "//": currentValue = self.floorDiv(currentValue, upcomingValue)
                case _ :   raise Exception(f"Error: Unknown operation '{currentToken.value}' at line {currentToken.position[0]}, character {currentToken.position[1]}")


        self.updateParentVariables(newParentVariables)

        return float(currentValue)

In [331]:
class FluoryBool(FluoryVariable):
    def __init__(self, id, value):
        super().__init__(id, value, "bool")

    def resolve(self, tokens):
        global Variables

        # We will need to keep track on what other variables we are reliant on
        newParentVariables = []

        currentValue = 0
        print(tokens[0])
        # First value must be a value
        if tokens[0].tokenType in ["int", "float", "str", "bool"]:
            currentValue = bool(tokens[0].value)

        # Turns out it will be a value soon
        elif tokens[0].value == "(":
            cursor, separatedTokens = self.separateTokensOn(")", 1, tokens)
            currentValue = self.resolve(separatedTokens)

        # Maybe its a value, its just hiding
        elif tokens[0].tokenType == "variableCall":
            if tokens[0].value in Variables:
                otherVar = Variables[tokens[0].value]

                currentValue = Variables[tokens[0].value].toBool()
                newParentVariables.append(tokens[0].value)

            else:
                raise Exception(f"Error: Variable '{tokens[0].value}' is undeclared")

        # Ok, its not a value after all
        else:
            raise Exception(f"Error: Expected boolean at line {tokens[0].position[0]}, character {tokens[0].position[1]}")

        cursor=0

        while cursor+1 < len(tokens):
            #This should be our next number
            cursor, currentToken = self.readNextToken(cursor, tokens)
            nextToken = self.peakNextToken(cursor, tokens)

            # Verifying that current value is indded an operation
            if currentToken.tokenType != "booleanOperation":
                raise Exception(f"Error: Expected boolean operation but got '{currentToken.tokenType}' at line {currentToken.position[0]}, character {currentToken.position[1]}")

            # Init number int
            upcomingValue = 0

            # Is it a number?
            if nextToken.tokenType in ["int", "float", "str", "bool"]:
                upcomingValue = bool(nextToken.value)
                cursor+=1

            # Its going to be a number!!
            elif nextToken.value == "(":
                cursor, separatedTokens = self.separateTokensOn(")", cursor+2, tokens)
                upcomingValue = self.resolve(separatedTokens)

            # Maybe its a number, its just hiding
            elif nextToken.tokenType == "variableCall":
                if nextToken.value in Variables:
                    otherVar = Variables[nextToken.value]

                    upcomingValue = otherVar.toBool()
                    newParentVariables.append(nextToken.value)
                    cursor+=1

                else:
                    raise Exception(f"Error: Variable '{nextToken.value}' is undeclared")

            # Turns out it was not a number after all..
            else:
                raise Exception(f"Error: Expected boolean but got '{nextToken.tokenType}' at line {tokens[0].position[0]}, character {tokens[0].position[1]}")

            # Doing math!
            match currentToken.value:
                case "&":  currentValue = self.andLogic(currentValue, upcomingValue)
                case "|":  currentValue = self.orLogic(currentValue, upcomingValue)
                case "^":  currentValue = self.xorLogic(currentValue, upcomingValue)

                case _ :   raise Exception(f"Error: Unknown operation '{currentToken.value}' at line {currentToken.position[0]}, character {currentToken.position[1]}")


        self.updateParentVariables(newParentVariables)

        return bool(currentValue)

In [332]:
class FluoryString(FluoryVariable):
    def __init__(self, id, value):
        super().__init__(id, value, "string")

    def resolve(self, tokens):
        global Variables

        # We will need to keep track on what other variables we are reliant on
        newParentVariables = []

        cursor=0
        currentValue = ""
        currentToken = tokens[cursor]

        while cursor < len(tokens):


            # Is it variable?
            if currentToken.tokenType == "variableCall":
                if currentToken.value in Variables:
                    currentValue += Variables[currentToken.value]
                    newParentVariables.append(currentToken.value)

                else:
                    raise Exception(f"Error: Variable '{currentToken.value}' is undeclared")

            # String interpolation. No questions asked. I'm lazy..
            else:
                currentValue += str(currentToken.value)

            # This should be our next value
            if cursor+1 >= len(tokens):
                break

            cursor, currentToken = self.readNextToken(cursor, tokens)


        self.updateParentVariables(newParentVariables)

        return str(currentValue)

In [333]:
# def getNewBranchID(self):
#         global HighestBranchId
#         HighestBranchId += 1
#         return HighestBranchId



# Tokenization

In [334]:
specialTokenKeys = [
    {
        "tokenKey": "arithmeticOperation",
        "tokens": [
            "+",
            "-",
            "*",
            "/",
            "%",
            "**",
            "//"
        ]
    },

    {
        "tokenKey": "comparisonOperation",
        "tokens": [
            "<",
            ">",
            "<=",
            ">=",
            "==",
            "!="
        ]
    },

    {
        "tokenKey": "assignmentOperation",
        "tokens": [
            "="
            # These do not make any sense in this context as they create a single variable infinite loop
            # "+=",
            # "-=",
            # "*=",
            # "/=",
            # "%=",
            # "**=",
            # "//=",
            # "|=",
            # "&=",
            # "^=",
            # "!!="
        ]
    },

    {
        "tokenKey": "booleanOperation",
        "tokens": [
            "!",
            "&",
            "|",
            "^"
        ]
    }
]

# These will break as if there's a space
UnifiedCharacters = [ "=", "+", "-", "*", "/", "%", "<", ">", "=", "!", "&", "|", "^" ]

for i in range(len(specialTokenKeys)):
    specialTokenKeys[i]["maxKeywordLength"] = len(max(specialTokenKeys[i]["tokens"], key = len))

# managing scope
scopeTokenKeys = [
    "(",
    ")",
    "{",
    "}",
    "[",
    "]",
    ";"
]

# variable types
variableTokenKeys = [
    "int",
    "float",
    "str",
    "bool"
]

# action keys
actionTokenKeys = [
    "if",
    "elif",
    "else",
    "for",
    "while",
    "break",
    "continue",
    "return"
]

In [335]:
def tokenize(code):
    tokens = []
    position = 0

    charPostition = 0
    linePosition = 1
    cursorPosition = -1

    cursorInComment = False
    cursorInMultilineComment = False


    while cursorPosition < len(code)-1:
        cursorPosition += 1
        currentChar = code[cursorPosition]

        if currentChar == "\n":
            linePosition += 1

        charPostition = 1

        # Skip whitespaces
        if currentChar.isspace():
            continue

        # Are we in a comment?
        if cursorInComment:
            if currentChar == "\n":
                cursorInComment = False
            else:
                continue

        # Are we in a multiline comment?
        elif cursorInMultilineComment:
            if currentChar == "#" and code[cursorPosition+1] == "#":
                cursorPosition += 1
                cursorInMultilineComment = False

            continue

        # Are we entering a comment?
        if currentChar == "#":

            # Multiline comment?

            if code[cursorPosition+1] == "#":
                cursorInMultilineComment = True
            else:
                cursorInComment = True

            continue

        # Special tokens
        if currentChar in UnifiedCharacters:
            currentValue = currentChar

            foundSpecialToken = False
            startingPosition = [linePosition, charPostition]

            while True:
                if cursorPosition+1 >= len(code):
                    raise Exception(f"Error: Unexpected end of file at line {linePosition}, character {charPostition}")

                if not (code[cursorPosition+1] in UnifiedCharacters):
                    break

                currentValue += code[cursorPosition+1]
                cursorPosition += 1

            for specialTokenKey in specialTokenKeys:

                if currentValue in specialTokenKey["tokens"]:
                    tokens.append(Token(specialTokenKey["tokenKey"], currentValue, startingPosition))
                    foundSpecialToken = True
                    break

            if not foundSpecialToken:
                raise Exception(f"Error: Unexpected character '{currentValue}' at line {startingPosition[0]}, character {startingPosition[1]}")

            continue

        # managing scope
        if currentChar in scopeTokenKeys:
            tokens.append(Token("scope", currentChar))
            continue

        # Are we in a string?
        # to be tested
        if currentChar == "\"":
            currentValue = ""
            cursorPosition += 1

            while True:
                if cursorPosition >= len(code):
                    raise Exception(f"Error: Unexpected end of file at line {linePosition}, character {charPostition}")

                if code[cursorPosition] == "\\" and code[cursorPosition+1] == "\"":
                    currentValue += "\""
                    cursorPosition += 2
                    charPostition += 2
                    continue

                if code[cursorPosition] == "\n":
                    linePosition += 1
                    currentValue += code[cursorPosition]

                if code[cursorPosition] == "\"":
                    cursorPosition += 1
                    charPostition += 1
                    break

                currentValue += code[cursorPosition]

                cursorPosition += 1
                charPostition += 1

            tokens.append(Token("str", currentValue, [linePosition, charPostition]))
            continue

        # Are we in a number?
        if currentChar.isdigit():
            currentvalue = currentChar
            floatFound = False

            while True:
                # Unexpected eof
                if cursorPosition+1 >= len(code):
                    break

                # still a digit
                if code[cursorPosition+1].isdigit():
                    cursorPosition += 1
                    charPostition += 1
                    currentvalue += code[cursorPosition]

                # now its a float
                elif code[cursorPosition+1] == ".":
                    # turns out its too much of a float
                    if floatFound:
                        raise Exception(f"Error: Unexpected character '.' at line {linePosition}, character {charPostition}")
                    floatFound = True
                    cursorPosition += 1
                    charPostition += 1
                    currentvalue += code[cursorPosition]

                # No longer a number
                elif code[cursorPosition+1] == " " or code[cursorPosition+1] == "\n" or code[cursorPosition+1] in UnifiedCharacters or code[cursorPosition+1] in scopeTokenKeys:
                    break

                # Missing space?
                else:
                    raise Exception(f"Error: Unexpected character '{code[cursorPosition+1]}' at line {linePosition}, character {charPostition}")

            negativeMultiplier = 1

            # To be simplified
            if len(tokens)>0 and tokens[len(tokens)-1].value == "-":
                if len(tokens)>1:
                    if not (tokens[len(tokens)-2].tokenType in ["int", "float", "str", "bool"]):
                        negativeMultiplier = -1
                        tokens.pop()

                else:

                    negativeMultiplier = -1
                    tokens.pop()

            if floatFound:
                tokens.append(Token("float", float(currentvalue)*negativeMultiplier, [linePosition, charPostition]))
            else:
                tokens.append(Token("int", int(currentvalue)*negativeMultiplier, [linePosition, charPostition]))

            continue

        if currentChar.isalpha():
            currentValue = currentChar

            while True:
                if cursorPosition+1 >= len(code):
                    raise Exception(f"Error: Unexpected end of file at line {linePosition}, character {charPostition}")

                if code[cursorPosition+1].isalpha():
                    cursorPosition += 1
                    charPostition += 1
                    currentValue += code[cursorPosition]

                elif code[cursorPosition+1] == "(":
                    tokens.append(Token("functionCall", currentValue))
                    break

                # this is a variable call
                elif code[cursorPosition+1] in UnifiedCharacters:
                    tokens.append(Token("variableCall", currentValue))
                    break

                # No longer this token
                elif code[cursorPosition+1].isspace() or code[cursorPosition+1] in scopeTokenKeys:
                    if currentValue in variableTokenKeys:
                        tokens.append(Token("variableDeclaration", currentValue))

                    # NO FUNCTIONS !!
                    # elif currentValue == "func":
                    #     tokens.append(Token("functionDeclaration", currentValue))

                    elif currentValue == "true" or currentValue == "false":
                        if(currentValue == "true"):
                            tokens.append(Token("bool", True))
                        else:
                            tokens.append(Token("bool", False))


                    elif currentValue in actionTokenKeys:
                        tokens.append(Token("action", currentValue))

                    else:
                        tokens.append(Token("variableCall", currentValue))

                    break

                # Missing space?
                else:
                    raise Exception(f"Error: Unexpected character '{code[cursorPosition+1]}' at line {linePosition}, character {charPostition}")

    # tokens.append(Token("eof", "eof"))

    return tokens

# Building AST (Abstract Syntax Tree)

In [341]:
def ast(text):
    global Variables

    Variables = {}

    def attemptNextToken(cursor, tokens):
        if cursor+1 >= len(tokens):
            raise Exception(f"Error: Unexpected end of file at line {tokens[cursor].position[0]}, character {tokens[cursor].position[1]}")

        return cursor+1, tokens[cursor+1]

    def getVariable(cursor, tokens):
        cursor, token = attemptNextToken(cursor, tokens)

        if token.tokenType != "variableCall":
            raise Exception(f"Error: Unexpected character '{token.value}' instead of variable name at line {token.position[0]}, character {token.position[1]}")

        varName = token.value

        cursor, token = attemptNextToken(cursor, tokens)

        if tokens[cursor].value != "=":
            raise Exception(f"Error: Unexpected character '{tokens[cursor].value}' instead of '=' at line {tokens[cursor].position[0]}, character {tokens[cursor].position[1]}")

        expression = []
        while cursor+1 < len(tokens) and tokens[cursor+1].value != ";":
            cursor, token = attemptNextToken(cursor, tokens)
            expression.append(token)

        return varName, expression, cursor


    tokens = tokenize(text)

    cursor = 0

    while cursor+1 < len(tokens):

        if tokens[cursor].tokenType == "variableDeclaration":
            match tokens[cursor].value:
                case "int":
                    varName, expression, cursor = getVariable(cursor, tokens)

                    if varName in Variables:
                        raise Exception(f"Error: Variable '{varName}' already declared at line {tokens[cursor].position[0]}, character {tokens[cursor].position[1]}")
                    Variables[varName] = FluoryInteger(varName, expression)
                    Variables[varName].resolveSelf()
                    Variables[varName].updateChildVariables()

                case "float":
                    varName, expression, cursor = getVariable(cursor, tokens)

                    if varName in Variables:
                        raise Exception(f"Error: Variable '{varName}' already declared at line {tokens[cursor].position[0]}, character {tokens[cursor].position[1]}")
                    Variables[varName] = FluoryFloat(varName, expression)
                    Variables[varName].resolveSelf()
                    Variables[varName].updateChildVariables()

                case "bool":
                    varName, expression, cursor = getVariable(cursor, tokens)

                    if varName in Variables:
                        raise Exception(f"Error: Variable '{varName}' already declared at line {tokens[cursor].position[0]}, character {tokens[cursor].position[1]}")
                    Variables[varName] = FluoryBool(varName, expression)
                    Variables[varName].resolveSelf()
                    Variables[varName].updateChildVariables()

                case "str":
                    varName, expression, cursor = getVariable(cursor, tokens)

                    if varName in Variables:
                        raise Exception(f"Error: Variable '{varName}' already declared at line {tokens[cursor].position[0]}, character {tokens[cursor].position[1]}")
                    Variables[varName] = FluoryString(varName, expression)
                    Variables[varName].resolveSelf()
                    Variables[varName].updateChildVariables()

        elif tokens[cursor].tokenType == "variableCall":
            varName, expression, cursor = getVariable(cursor-1, tokens)

            Variables[varName].expression = expression
            Variables[varName].resolveSelf()
            Variables[varName].updateChildVariables()


        elif tokens[cursor].tokenType == "functionCall":
            if tokens[cursor].value == "print":
                cursor, token = attemptNextToken(cursor, tokens)

                if token.value != "(":
                    raise Exception(f"Error: Unexpected character '{token.value}' instead of '(' at line {token.position[0]}, character {token.position[1]}")

                cursor, token = attemptNextToken(cursor, tokens)

                if token.tokenType not in ["variableCall", "str", "bool", "int", "float"]:
                    raise Exception(f"Error: Unexpected character '{token.value}' instead of variable name at line {token.position[0]}, character {token.position[1]}")

                if token.tokenType == "variableCall":
                    if token.value not in Variables:
                        raise Exception(f"Error: Variable '{token.value}' not declared at line {token.position[0]}, character {token.position[1]}")

                    print(Variables[token.value])
                else:
                    print(token.value)

                cursor, token = attemptNextToken(cursor, tokens)
                if token.value != ")":
                    raise Exception(f"Error: Unexpected character '{token.value}' instead of ')' at line {token.position[0]}, character {token.position[1]}")


        cursor += 1

# Playarea

In [337]:
# still int needs space at the end for some reason.
# did not think about negative numbers
innerCode = "- 2"
tokens = tokenize(innerCode)
myInt = FluoryInteger("testy", tokenize(innerCode))
myInt.resolveSelf()
myInt.updateChildVariables()

print(myInt)

# innerCode = "testy&true "
# tokens = tokenize(innerCode)
# myInt = FluoryBool("test", tokenize(innerCode))
# myInt.resolveSelf()
# myInt.updateChildVariables()

# print(myInt)

# innerCode = "true "
# tokens = tokenize(innerCode)
# Variables["testy"].expression = tokens
# Variables["testy"].resolveSelf()
# Variables["testy"].updateChildVariables()

# print(Variables["testy"])

# print(Variables["test"])


-2


In [343]:
ast("""
int test = 2 ;
int testy = 3+test;
print(testy) ;

test = 6 ;
print(testy) ;
""")



scope ; [1, 1]
scope ; [1, 1]
5
scope ; [1, 1]
scope ; [1, 1]
9
